# Pandas vs Polars API Comparison: Code Examples

This notebook provides the Python code examples discussed in the accompanying article comparing the APIs of the Pandas and Polars DataFrame libraries. It demonstrates syntax for common data manipulation tasks and includes the illustrative benchmark code.

Make sure you have both libraries installed:

In [ ]:
pip install pandas polars

Let's start with the necessary imports.

In [ ]:
import pandas as pd
import polars as pl
import time
import gc # Garbage collector
import numpy as np # For introducing nulls in Pandas example

print(f"Pandas version: {pd.__version__}")
print(f"Polars version: {pl.__version__}")

Now, let's compare the Pandas API and Polars API side-by-side for essential data manipulation tasks. We'll use illustrative code snippets, assuming you have imported the libraries as pd and pl respectively. 

Note: The following code placeholders assume DataFrames like `df_pandas`, `df_polars`, `df_left_pandas`, `df_right_polars`, etc., have been defined. You will need to define these yourself based on the context or previous examples if you wish to run these cells directly.

### Reading/Writing Data (CSV Example)  

Loading data from files and writing results back is a fundamental step.

**Pandas**

Uses straightforward functions like `read_csv()`. Execution is eager.

In [ ]:
# Reading (Eager)
df_pandas = pd.read_csv("data.csv")

# Writing
df_pandas.to_csv("output_pandas.csv", index=False) # Must often disable index writing

**Polars**

Offers both eager `read_csv()` and lazy `scan_csv()` options. Lazy scanning is highly recommended for large files as it allows optimizations before loading data into memory.

In [ ]:
# Reading (Eager)
df_polars = pl.read_csv("data.csv")

# Reading (Lazy - Preferred for large files)
lf_polars = pl.scan_csv("data.csv")
# ... other lazy operations on lf_polars ...
df_polars = lf_polars.collect() # Execute plan and load data

# Writing
df_polars.write_csv("output_polars.csv")
# LazyFrames can also sink directly
# lf_polars.sink_csv("output_lazy.csv")

### Selection (Rows & Columns)
Selecting specific subsets of your data is a core operation.

**Pandas** 

Offers flexible selection using `[]`, label-based `.loc[]`, and integer position based `.iloc[]`.

In [ ]:
# Select single column 'A'
col_a_pandas = df_pandas['A']

# Select multiple columns
subset_cols_pandas = df_pandas[['A', 'B']]

# Select rows by integer position (slicing)
subset_rows_pandas = df_pandas.iloc[5:10]

# Select rows and columns by label/position
subset_loc_pandas = df_pandas.loc[df_pandas['index_col'] == 'label', ['A', 'B']] # Using .loc
subset_iloc_pandas = df_pandas.iloc[5:10, [0, 2]] # Using .iloc

**Polars**


Uses `select()` for columns, `filter()` for row conditions (see next section), and `[]` for row slicing or selecting columns by name.

In [ ]:
# Select single column 'A' (returns Series)
col_a_polars = df_polars['A']
# More explicit way using select (returns DataFrame)
col_a_df_polars = df_polars.select(pl.col('A'))

# Select multiple columns
subset_cols_polars = df_polars.select(['A', 'B'])
# Using expressions
subset_cols_expr_polars = df_polars.select(pl.col('A'), pl.col('B'))

# Select rows by integer position (slicing)
subset_rows_polars = df_polars[5:10]

# Select rows and columns (usually involves chaining filter/select or slicing)
subset_filter_select_polars = df_polars.filter(pl.col('C') > 10).select(['A', 'B'])
subset_slice_select_polars = df_polars[5:10].select(['A', 'B'])

### Filtering Data
Selecting rows based on conditions is essential for analysis.

**Pandas**

Commonly uses boolean masking or the `.query()` method.

In [ ]:
# Boolean masking
filtered_pandas = df_pandas[df_pandas['A'] > 100]
filtered_multi_pandas = df_pandas[(df_pandas['A'] > 100) & (df_pandas['B'] == 'category1')]

# Using .query()
filtered_query_pandas = df_pandas.query("A > 100 and B == 'category1'")

**Polars**

Uses the `filter()` method with expressions.

In [ ]:
# Using filter() with expressions
filtered_polars = df_polars.filter(pl.col('A') > 100)
filtered_multi_polars = df_polars.filter(
    (pl.col('A') > 100) & (pl.col('B') == 'category1')
)

### Creating/Modifying Columns
Adding new columns or changing existing ones based on calculations.

**Pandas**

Direct assignment (`df['new_col'] = ...`) is common; `.assign()` provides a method-chaining approach.

In [ ]:
# Direct assignment
df_pandas['C'] = df_pandas['A'] * 10
df_pandas['D'] = df_pandas['A'] / df_pandas['B'] # Assumes numeric B

# Using .assign()
df_pandas = df_pandas.assign(
    C = df_pandas['A'] * 10,
    D = lambda x: x['A'] / x['B'] # Can use functions
)

**Polars**

Uses `with_columns()` which takes a list of expressions. Each expression typically defines the calculation and uses `.alias()` to name the new/modified column.

In [ ]:
# Using with_columns()
df_polars = df_polars.with_columns([
    (pl.col('A') * 10).alias('C'),
    (pl.col('A') / pl.col('B')).alias('D') # Assumes numeric B
])

# Creating multiple columns, including conditional logic
df_polars = df_polars.with_columns([
    (pl.col('A') * 10).alias('C'),
    pl.when(pl.col('A') > 100)
      .then(pl.lit("High")) # pl.lit() for literal values
      .otherwise(pl.lit("Low"))
      .alias('A_category')
])

### Grouping and Aggregation
Summarizing data by groups is a cornerstone of analysis.

**Pandas**

Uses the `groupby().agg()` pattern, often specifying aggregation functions as strings or using named aggregation.

In [ ]:
# Group by 'group_col', calculate mean of 'A' and max of 'B'
agg_pandas = df_pandas.groupby('group_col').agg(
    avg_A = ('A', 'mean'),
    max_B = ('B', 'max')
)

**Polars**

Uses a similar `group_by().agg()` structure, but aggregations are defined using expressions.

In [ ]:
# Group by 'group_col', calculate mean of 'A' and max of 'B'
agg_polars = df_polars.group_by('group_col').agg([
    pl.mean('A').alias('avg_A'), # Use Polars aggregation functions
    pl.max('B').alias('max_B'),
    pl.count().alias('group_size'), # Count rows in each group
    pl.first('C').alias('first_C_in_group') # Get first value
])

### Joining/Merging DataFrames
Combining data from multiple sources based on common keys.

**Pandas**

Uses the `pd.merge()` function or the DataFrame's `.join()` method (which often requires setting the index on one DataFrame).

In [ ]:
# Assume df_left_pandas, df_right_pandas exist
merged_pandas = pd.merge(df_left_pandas, df_right_pandas, on='key_col', how='inner')

# Using .join() - often needs index alignment
# joined_pandas = df_left_pandas.join(df_right_pandas.set_index('key_col'), on='key_col', how='left')

**Polars**

Uses a single, powerful `.join()` method.

In [ ]:
# Assume df_left_polars, df_right_polars exist
joined_polars = df_left_polars.join(df_right_polars, on='key_col', how='inner')

# Different join keys, left join
joined_left_polars = df_left_polars.join(
    df_right_polars,
    left_on='left_key',
    right_on='right_key',
    how='left'
)

### Handling Missing Data
Dealing with null or NaN values. (Requires DataFrames with nulls introduced manually).

**Pandas**

Uses `.isnull()`, `.fillna()`, `.dropna()`.

In [ ]:
# Check for nulls
nulls_pandas = df_pandas['A'].isnull()

# Fill nulls
filled_pandas = df_pandas.fillna(0) # Fill all with 0
filled_specific_pandas = df_pandas.fillna({'A': 0, 'B': 'Unknown'})

# Drop rows with any nulls
dropped_pandas = df_pandas.dropna()

**Polars**

Uses analogous methods `.is_null()`, `.fill_null()`, `.drop_nulls()`.

In [ ]:
# Check for nulls (creates boolean Series/Expression)
nulls_polars = df_polars['A'].is_null()
# In an expression context: pl.col('A').is_null()

# Fill nulls
filled_polars = df_polars.fill_null(0) # Fill all with 0
# Fill specific columns or use strategies (expressions)
filled_strategy_polars = df_polars.with_columns([
    pl.col('A').fill_null(0),
    pl.col('B').fill_null(pl.lit('Unknown')),
    pl.col('C').fill_null(pl.median('C')) # Fill with column median
])

# Drop rows with any nulls
dropped_polars = df_polars.drop_nulls()

### Applying Custom Functions
For operations not covered by built-in functions, applying custom Python logic.

**Pandas**

Uses `.apply()` (can be row-wise or column-wise, often slow for rows) or `.map()` (element-wise on Series).

In [ ]:
# Apply row-wise (use cautiously - performance)
df_pandas['custom_result'] = df_pandas.apply(lambda row: row['A'] + row['B'] if row['C'] else row['A'], axis=1)

# Map element-wise
df_pandas['A_mapped'] = df_pandas['A'].map(lambda x: x**2)

**Polars**

Provides `.map_elements()` for element-wise operations (faster than `.apply`, requires dtype specification) and `.apply()` (generally slow, breaks optimizations). Crucially, Polars strongly encourages using its built-in expressions whenever possible for performance.

In [ ]:
# *** STRONGLY PREFER BUILT-IN EXPRESSIONS ***
# Example equivalent to the pandas apply lambda using expressions:
df_polars = df_polars.with_columns(
    pl.when(pl.col('C'))
      .then(pl.col('A') + pl.col('B'))
      .otherwise(pl.col('A'))
      .alias('custom_result_expr')
)

# Map element-wise (use only if no expression exists)
df_polars = df_polars.with_columns(
    pl.col('A').map_elements(lambda x: x**2, return_dtype=pl.Float64).alias('A_mapped')
    # Need return_dtype, potentially slower than expressions
)

# Apply row-wise (AVOID if possible - significant performance cost)
# df_polars['custom_apply'] = df_polars.apply(lambda row_dict: ...) # Slow!

### Illustrative Benchmark Example (Iowa Sales Data)
This section contains the benchmark code comparing Pandas and Polars on the Iowa Liquor Sales dataset. It performs reading, cleaning ('Sale (Dollars)'), grouping ('County'), and aggregation (mean sales).

Note: Running this requires downloading the dataset (`Iowa_Liquor_Sales.csv`) and placing it at the specified path (`/kaggle/input/iowa-liquor-sales/Iowa_Liquor_Sales.csv` or adjusting the path). Results depend heavily on your environment. In a specific execution run shared, Polars completed the task in approximately 9.6 seconds, whereas Pandas took around 108.88 seconds – showcasing a dramatic difference for this workload.

**Polars**

In [ ]:
import polars as pl
import time
import gc # Garbage collector

# --- Polars Example: Iowa Liquor Sales Aggregation ---
# Reads the data, cleans the sales column, groups by county, and calculates mean sales.

print("--- Running Polars Iowa Sales Aggregation Example ---")

# Define the path to your data file
# Make sure this path is correct for your environment
csv_file_path = "/kaggle/input/iowa-liquor-sales/Iowa_Liquor_Sales.csv"

# Define relevant column names
sales_col = 'Sale (Dollars)'
county_col = 'County'
avg_sales_col = 'Average Sale (Dollars)' # Name for the aggregated column

try:
    # Record start time
    start_time = time.time()

    # --- Build the Polars Lazy Query ---
    # 1. Scan the CSV lazily
    lf = pl.scan_csv(csv_file_path)

    # 2. Clean the 'Sale (Dollars)' column and cast to Float64
    #    This expression overwrites the original column
    lf = lf.with_columns(
        pl.col(sales_col)
          .cast(pl.Utf8)           # Ensure it's string type first
          .str.replace(r"\$", "")  # Remove the '$' character
          .cast(pl.Float64)       # Cast the cleaned string to Float64
    )

    # 3. Group by 'County' and calculate the mean of the cleaned 'Sale (Dollars)'
    lf_agg = lf.group_by(county_col).agg(
        pl.mean(sales_col).alias(avg_sales_col) # Calculate mean and rename
    )

    # 4. Execute the entire lazy plan
    result_df = lf_agg.collect()

    # Record end time
    end_time = time.time()
    duration = end_time - start_time

    # --- Output Results ---
    print(f"Polars operation took: {duration:.4f} seconds")
    print("\nAggregation Result (Top 5):")
    print(result_df.head()) # Display the first few rows of the result

    # --- Clean up memory ---
    del lf # LazyFrame reference
    del lf_agg # LazyFrame reference
    del result_df # DataFrame reference
    gc.collect()

# --- Error Handling ---
except FileNotFoundError:
    print(f"\nError: Data file not found at {csv_file_path}")
    print("Please ensure the file path is correct.")
except Exception as e:
    # Catch other potential errors during processing (e.g., column not found)
    print(f"\nAn error occurred during Polars processing: {e}")

In [ ]:
import pandas as pd
import time
import gc # Garbage collector

# --- Pandas Example: Iowa Liquor Sales Aggregation ---
# Reads the data, cleans the sales column, groups by county, and calculates mean sales.
# Note: Pandas operations are typically eager.

print("--- Running Pandas Iowa Sales Aggregation Example ---")

# Define the path to your data file
# Make sure this path is correct for your environment
csv_file_path = "/kaggle/input/iowa-liquor-sales/Iowa_Liquor_Sales.csv"

# Define relevant column names
sales_col = 'Sale (Dollars)'
county_col = 'County'
avg_sales_col = 'Average Sale (Dollars)' # Name for the aggregated column

try:
    # Record start time
    start_time = time.time()

    # --- Perform Pandas Operations Eagerly ---
    # 1. Read the entire CSV into memory
    df = pd.read_csv(csv_file_path)

    # 2. Clean the 'Sale (Dollars)' column and cast to float
    #    Ensure the column exists before attempting cleaning
    if sales_col in df.columns:
        # Remove '$' and convert to numeric (float)
        # Using errors='coerce' will turn problematic values into NaN
        df[sales_col] = pd.to_numeric(
            df[sales_col].astype(str).str.replace(r'\$', '', regex=True),
            errors='coerce'
        )
        # Alternative using .astype after replace:
        # df[sales_col] = df[sales_col].astype(str).str.replace(r'\$', '', regex=True).astype(float)
    else:
        raise ValueError(f"Column '{sales_col}' not found in the CSV.")

    # 3. Group by 'County' and calculate the mean of the cleaned 'Sale (Dollars)'
    # Ensure the county column exists
    if county_col in df.columns:
         # Group, aggregate, and reset index to make 'County' a column again
        result_df = df.groupby(county_col)[sales_col].mean().reset_index()
        # Rename the aggregated column for clarity
        result_df = result_df.rename(columns={sales_col: avg_sales_col})
    else:
         raise ValueError(f"Column '{county_col}' not found in the CSV.")

    # Record end time
    end_time = time.time()
    duration = end_time - start_time

    # --- Output Results ---
    print(f"Pandas operation took: {duration:.4f} seconds")
    print("\nAggregation Result (Top 5):")
    print(result_df.head()) # Display the first few rows of the result

    # --- Clean up memory ---
    del df # DataFrame reference
    del result_df # DataFrame reference
    gc.collect()

# --- Error Handling ---
except FileNotFoundError:
    print(f"\nError: Data file not found at {csv_file_path}")
    print("Please ensure the file path is correct.")
except ValueError as ve:
    # Catch specific errors like missing columns
    print(f"\nData Error: {ve}")
except Exception as e:
    # Catch other potential errors during processing
    print(f"\nAn error occurred during Pandas processing: {e}")


This notebook provides a practical reference for the code used in the Pandas vs Polars API comparison. Remember that the best choice of library depends on your specific needs regarding performance, data size, and ecosystem integration.